In [1]:
# mecab 설치 방법
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 12.88 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 684 kB/s 
     |████████████████████████████████| 465 kB 59.3 MB/s 
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-12-03 10:25:26--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::3403:4be7, 2406:da00:ff00::6b17:d1f5, ...
Connecting to bitbucket.org (b

In [2]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
#케라스 2.2설치
!pip install q keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from collections import Counter
from konlpy.tag import Mecab
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense

In [5]:
df_raw = pd.read_csv('./raw_data.csv', encoding="CP949")
df_raw.head()

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (22,27,39,41,44,45) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,번호,개방서비스명,개방서비스아이디,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태구분코드,영업상태명,상세영업상태코드,...,공장판매직직원수,공장생산직직원수,건물소유구분명,보증액,월세액,다중이용업소여부,시설총규모,전통업소지정번호,전통업소주된음식,홈페이지
0,1,일반음식점,07_24_04_P,3220000,3220000-101-2022-00618,20220527,NaN,3,폐업,2,...,0.0,0.0,NaN,0.0,0.0,N,0.00,NaN,NaN,NaN
1,2,일반음식점,07_24_04_P,4090000,4090000-101-2022-00280,20220519,NaN,3,폐업,2,...,0.0,0.0,NaN,0.0,0.0,N,15.00,NaN,NaN,NaN
2,3,일반음식점,07_24_04_P,5670000,5670184-101-2022-00048,20220428,NaN,3,폐업,2,...,0.0,0.0,NaN,0.0,0.0,N,114.95,NaN,NaN,NaN
3,4,일반음식점,07_24_04_P,4200000,4200000-101-2022-00132,20220524,NaN,3,폐업,2,...,0.0,0.0,NaN,0.0,0.0,N,0.00,NaN,NaN,NaN
4,5,일반음식점,07_24_04_P,4200000,4200000-101-2022-00131,20220523,NaN,3,폐업,2,...,0.0,0.0,NaN,0.0,0.0,N,0.00,NaN,NaN,NaN


In [6]:
df = df_raw[["사업장명", "위생업태명"]]


In [7]:
df = df.dropna()
df = df.drop_duplicates(["사업장명", "위생업태명"])


In [8]:
sector = dict()
sector['한식'] = ["냉면집", "한식", "탕류(보신용)", "식육(숯불구이)"]
sector['일식'] = ["횟집", "북어취급", "일식", "경양식"]
sector['중식'] = ["중국식", "외국음식전문점(인도,태국등)"]
sector['카페'] = ["다방", "까페", "커피숍", "전통찻집", "키즈카페", "제과점영업"]
sector['패스트푸드'] = ["통닭(치킨)", "호프/통닭", "패스트푸드", "분식"]

other = ["이동조리", "일반조리판매", "출장조리", "패밀리레스트랑", "뷔페식", "식품등 수입판매업", "식품소분업", "기타", "간이주점", "소주방", "감성주점", "정종/대포집/소주방"]

df = df[df["위생업태명"].isin(other) == False]
df

,사업장명,위생업태명
0,팝업스토어(한시적),한식
1,꼬꼬장,호프/통닭
3,2022 강릉단오제 먹거리존(관음사포교당),한식
12,2022 강릉단오제 먹거리존,한식
17,아뚜르캠핑,한식
...,...,...
1048569,삼화만두우동,한식
1048570,아쿠아위에,한식
1048571,해피치즈스마일,한식
1048572,다원양분식,한식


In [9]:
def change(x):
  global sector
  for sector_i in sector.keys():
    if x in sector[sector_i]:
      return sector_i

df["위생업태명"] = df["위생업태명"].apply(change)

# 한국어만 가능(최대 외래어)
df["사업장명"] = df["사업장명"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


# 전처리 완료

df

<ipython-input-9-c092594cea4e>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df["사업장명"] = df["사업장명"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


,사업장명,위생업태명
0,팝업스토어한시적,한식
1,꼬꼬장,패스트푸드
3,강릉단오제 먹거리존관음사포교당,한식
12,강릉단오제 먹거리존,한식
17,아뚜르캠핑,한식
...,...,...
1048569,삼화만두우동,한식
1048570,아쿠아위에,한식
1048571,해피치즈스마일,한식
1048572,다원양분식,한식


In [10]:
# test 데이터와 train 데이터의 구분


train_data, test_data = train_test_split(df, test_size= 0.25, random_state = 42)
train_data.astype(str)

,사업장명,위생업태명
788536,노닭노닭,패스트푸드
147691,커피잇수다,카페
64412,삐아띠이탈리아니,일식
154758,윤가네남원추어탕,한식
933790,등등집별관,패스트푸드
...,...,...
161661,징기스칸이류점,패스트푸드
431625,이치코퍼레이션주카오카무,중식
650397,경은,한식
199287,거북산장,한식


In [12]:
# 학습 과정 1
# 단어 분리 Mecab으로 토큰화를 진행

mecab = Mecab()

train_data["tokenized"] = train_data["사업장명"].apply(mecab.morphs)
test_data['tokenized'] = test_data['사업장명'].apply(mecab.morphs)
train_data = train_data.dropna()
test_data = test_data.dropna()

In [13]:
train_data.isnull().sum()

사업장명         0
위생업태명        0
tokenized    0
dtype: int64

In [14]:
# 각 음식점에서 많이 나타나는 단어를 정리

hansik = np.hstack(train_data[train_data["위생업태명"]=="한식"]['tokenized'].values)
ilsik = np.hstack(train_data[train_data["위생업태명"]=="일식"]['tokenized'].values)
zongsik = np.hstack(train_data[train_data["위생업태명"]=="중식"]['tokenized'].values)
cafe = np.hstack(train_data[train_data["위생업태명"]=="카페"]['tokenized'].values)
fastfood = np.hstack(train_data[train_data["위생업태명"]=="패스트푸드"]['tokenized'].values)


In [15]:
# 데이터들을 tokenized된 단어들로 부터 사업장명을 에측할 수 있도록 함

X_train = train_data['tokenized'].values
y_train = train_data['위생업태명'].values
X_test= test_data['tokenized'].values
y_test = test_data['위생업태명'].values

In [16]:
# 정수 인코딩
tokenizer = Tokenizer(num_words=100)
tokenizer.fit_on_texts(X_train)

In [17]:
# 3번 이상 나타난 단어를 추산함
threshold = 3
total_cnt = len(tokenizer.word_index)
rare_cnt = 0

for key, value in tokenizer.word_counts.items():
  if (value < threshold):
    rare_cnt = rare_cnt + 1

# 3번 이상 나타나지 않은 것들에 대해서 활용 +2는 0, 1번 토큰을 고려함
vocab_size = total_cnt - rare_cnt + 2 
print(vocab_size)

23440


In [18]:
tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [19]:
X_train = pad_sequences(X_train, maxlen=5)
X_test = pad_sequences(X_test, maxlen=5)
X_train = np.array(X_train)
X_test = np.array(X_test)

encoder = LabelEncoder()
y_train_len = len(y_train)
y_test_len = len(y_test)
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)
y_train = y_train.reshape(y_train_len, 1)
y_test = y_test.reshape(y_test_len, 1)

# 일식, 중식, 카페, 패스트푸드, 한식

In [24]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout
from keras.layers import Flatten, Conv1D, GlobalMaxPooling1D

# CNN 모델의 활용
model = Sequential()
model.add(Embedding(vocab_size, 128, input_length=5))
model.add(Dropout(0.2))
model.add(Conv1D(256,
                 3,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))

In [25]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 5, 128)            3000320   
                                                                 
 dropout_2 (Dropout)         (None, 5, 128)            0         
                                                                 
 conv1d_1 (Conv1D)           (None, 3, 256)            98560     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 256)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                      

In [26]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=["SparseCategoricalAccuracy"])

In [27]:
model_classification = model.fit(X_train, y_train, epochs =15, batch_size=64)
loss_and_metrics = model.evaluate(X_test, y_test, batch_size=64)

Epoch 1/15
6448/6448 [==============================] - 35s 5ms/step - loss: 0.8895 - sparse_categorical_accuracy: 0.6515
Epoch 2/15
6448/6448 [==============================] - 36s 6ms/step - loss: 0.8233 - sparse_categorical_accuracy: 0.6793
Epoch 3/15
6448/6448 [==============================] - 35s 6ms/step - loss: 0.7868 - sparse_categorical_accuracy: 0.6940
Epoch 4/15
6448/6448 [==============================] - 35s 5ms/step - loss: 0.7552 - sparse_categorical_accuracy: 0.7056
Epoch 5/15
6448/6448 [==============================] - 36s 6ms/step - loss: 0.7287 - sparse_categorical_accuracy: 0.7151
Epoch 6/15
6448/6448 [==============================] - 35s 5ms/step - loss: 0.7051 - sparse_categorical_accuracy: 0.7244
Epoch 7/15
6448/6448 [==============================] - 37s 6ms/step - loss: 0.6848 - sparse_categorical_accuracy: 0.7314
Epoch 8/15
6448/6448 [==============================] - 35s 5ms/step - loss: 0.6664 - sparse_categorical_accuracy: 0.7388
Epoch 9/15
6448/6448 [==

In [39]:
# 모델의 정홗도
y = model.predict(X_test)
y = max_index(y)
print(classification_report(y_test, y, target_names=["일식", "중식", "카페", "패스트푸드", "한식"]))
cm = multilabel_confusion_matrix(y_test, y)
for i in range(len(cm)):
  print("%s 기준으로 이진분류한 오차행렬:\n"%i,
        pd.DataFrame(cm[i]).rename(index={0:"실제값(N)", 1:"실제값(P)"}, columns={0:"예측값(N)", 1:"예측값(P)"}))
  print("-"*30)

4299/4299 [==============================] - 7s 2ms/step
              precision    recall  f1-score   support

          일식       0.44      0.40      0.42     17794
          중식       0.59      0.38      0.46      4400
          카페       0.29      0.13      0.18      3315
       패스트푸드       0.55      0.59      0.57     39194
          한식       0.72      0.75      0.73     72843

    accuracy                           0.63    137546
   macro avg       0.52      0.45      0.47    137546
weighted avg       0.62      0.63      0.62    137546

0 기준으로 이진분류한 오차행렬:
         예측값(N)  예측값(P)
실제값(N)  110708    9044
실제값(P)   10591    7203
------------------------------
1 기준으로 이진분류한 오차행렬:
         예측값(N)  예측값(P)
실제값(N)  131993    1153
실제값(P)    2734    1666
------------------------------
2 기준으로 이진분류한 오차행렬:
         예측값(N)  예측값(P)
실제값(N)  133174    1057
실제값(P)    2880     435
------------------------------
3 기준으로 이진분류한 오차행렬:
         예측값(N)  예측값(P)
실제값(N)   79548   18804
실제값(P)   16146   23048
-----

In [30]:
def max_index(arr):
  new_arr = []
  for i in range(len(arr)):
    arr_one = arr[i]
    max_num = list(np.where(arr_one == max(arr_one))[0])
    new_arr.append(max_num)
  new_arr = np.array(new_arr)
  return new_arr

In [ ]:
# 예측할 수 있는 prediction 함수
# arr는 토큰화 되어 있는 요소를 의미함
def prediction(model, arr):
  series = ["일식", "중식", "카페", "패스트푸드", "한식"]
  get_num = [0,0,0,0,0]
  y_predicted = model.predict(arr)
  predictor = max_index(y_predicted)
  for i in range(len(predictor)):
    get_num[predictor[i][0]] += 1
  best_ated = get_num.index(max(get_num))
  print("당신의 취향은 {0}입니다.".format(series[best_ated]))

prediction(model, X_test) # 해당 취향을 출력하게 됨

In [ ]:
# OCR을 통해서 가져온 json 파일들을 변환
import json
import csv

# json 파일을 csv 파일로 변환(OCR을 통해서 json의 파일들로 만든다)
with open('receip.json', 'r', encoding = 'utf-8') as input_data, open('melon.csv', 'w', newline = '') as output_file :
  new_data = json.load(input_data)
  f = csv.writer(output_file)
  f.writerow(["name"])
  for data in new_data:
    f.writerow([data["result"]["storeInfo"]["name"]])

# csv의 name을 토큰화 시키는 방법
def change_input(new_data):
  mecab = Mecab()
  new_data["tokenized"] = new_data["name"].apply(mecab.morphs)
  new_data_input = new_data['tokenized'].values
  new_data_input = tokenizer.texts_to_sequences(new_data_input)
  new_data_input = pad_sequences(new_data_input, maxlen=5)
  new_data_input = np.array(new_data_input)
  return new_data_input

df_new = pd.read_csv('/content/drive/MyDrive/raw_data.csv', encoding="CP949")
df_new = change_input(df_new)
prediction(model, df_new)
